In [ ]:
%load_ext autoreload
%autoreload 2

import sys
# adding to the system path
sys.path.insert(0, '/home/prashanth/Thesis/Imitation-Learning/')

In [ ]:
import rospy
from Robot.sawyer import Sawyer
from Robot.ft_sensor import FTSensor

rospy.init_node('record_demos')
robot = Sawyer()
#ft_sensor = FTSensor(should_plot=False)

In [ ]:
robot.robot.move_to_neutral()

In [ ]:
robot.robot.enable_robot()

In [ ]:
robot.robot.endpoint_effort()

### Ergodic exploration using tensor train

In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture

file_path = 'data/gear/no_gear/1_ee_poses.npy'
recorded_traj = np.load(file_path, allow_pickle=True)
recorded_traj = recorded_traj[200:400, :]

gmm = GaussianMixture(n_components=2, random_state=0).fit(recorded_traj)

In [ ]:
gmm.covariances_

#### CollaborativeTransportation2D paper get reproduction data from position and dt.

In [ ]:
from Robot import kdl_utils
from Common import config
import numpy as np
import rospy

rate = rospy.Rate(config.CONTROL_RATE)
dt = rate.sleep_dur.to_sec()


####  parse Data and save to ´.mat´ file for VIC paper

In [ ]:
from scipy import io
from parser import vic_data_parser

mdict = vic_data_parser(path='data/gear/', dt=1/20)
path = '/home/prashanth/Thesis/Resources/Force/Codes/ras18_toy_example/data/'
io.savemat(path+"gear_data_01.mat", mdict)

####  parse Data and save to ´.mat´ file for Collabrative 2D paper

In [ ]:
from scipy import io
from parser import collaborative_planar

mdict = collaborative_planar(path='data/gear/no_gear/', dt=1/20)
path = '/home/prashanth/Thesis/Resources/Force/Codes/CollaborativeTransportation2D/data/3D/'
io.savemat(path+"gear_data_01.mat", mdict)

### Send commands to the Cartesian Impedence controller

In [ ]:
import matlab.engine
import rospy
rospy.init_node('cart_imp_control')
eng = matlab.engine.start_matlab()


In [ ]:
from geometry_msgs.msg import PoseStamped
import tf.transformations
import numpy as np
import rospy
from franka_msgs.msg import FrankaState


imp_control_pub = rospy.Publisher('/cartesian_impedance_example_controller/equilibrium_pose', PoseStamped, queue_size=10)

msg = rospy.wait_for_message("franka_state_controller/franka_states",
                                 FrankaState)  # type: FrankaState

link_name = 'panda_link0'
marker_pose = PoseStamped()

initial_quaternion = \
    tf.transformations.quaternion_from_matrix(
        np.transpose(np.reshape(msg.O_T_EE,
                                (4, 4))))
initial_quaternion = initial_quaternion / \
    np.linalg.norm(initial_quaternion)
marker_pose.pose.orientation.x = initial_quaternion[0]
marker_pose.pose.orientation.y = initial_quaternion[1]
marker_pose.pose.orientation.z = initial_quaternion[2]
marker_pose.pose.orientation.w = initial_quaternion[3]
marker_pose.header.frame_id = link_name
marker_pose.header.stamp = rospy.Time(0)
marker_pose.pose.position.x = msg.O_T_EE[12] 
marker_pose.pose.position.y = msg.O_T_EE[13] 
marker_pose.pose.position.z = msg.O_T_EE[14]

#imp_control_pub.publish(marker_pose)


In [ ]:
def publisher_callback(msg, link_name):
    marker_pose.header.frame_id = link_name
    marker_pose.header.stamp = rospy.Time(0)
    imp_control_pub.publish(marker_pose)

In [ ]:
rospy.Timer(rospy.Duration(0.01),
                lambda msg: publisher_callback(msg, link_name))

In [ ]:
curr_Pos = {}
def get_state(msg):
    curr_Pos['x'] = msg.O_T_EE[12]
    curr_Pos['y'] = msg.O_T_EE[13]
    curr_Pos['z'] = msg.O_T_EE[14]
franka_state_sub = rospy.Subscriber("franka_state_controller/franka_states", FrankaState, get_state)

### Move to B-pose using Impedence controller

In [ ]:
from Robot import kdl_utils
import numpy as np
import tf_conversions

bottleneck_pose_vector = np.load('data/bottleneck_pose_vector.npy', allow_pickle=True)
bottelneck_pose = kdl_utils.create_pose_from_vector(bottleneck_pose_vector)
pose_msg = tf_conversions.toMsg(bottelneck_pose)
marker_pose.pose.position = pose_msg.position

In [ ]:
!rostopic pub -1 /franka_ros_interface/franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

### Allign with the gear pin, since we are planning only 2d path i.e. x & z

In [ ]:
marker_pose.pose.position.y = -0.02720078072969254

In [ ]:
curr_Pos

In [ ]:
marker_pose.pose.position.x = 0.53638506
marker_pose.pose.position.y = -0.04493493
marker_pose.pose.position.z = 0.15896212

In [ ]:
from mat2py.functions import pHRI_toyExample_PlanarManip01

currPos = matlab.double([[curr_Pos['x']], [curr_Pos['y']], [curr_Pos['z']]])
matlab_data = pHRI_toyExample_PlanarManip01(eng, currPos, dt=np.arange(0.0, 2, 0.01 )) # [t(1), currPos(2), currVel(2), currAcc(2), tarTmp(2), force(2)]
matlab_data = np.squeeze(np.array(matlab_data))

In [ ]:
matlab_data.T[-1,1:4]

### Compute the whole trajectory at once

In [ ]:
rospy.sleep(8.0)
rate = rospy.Rate(100)
traj = np.load('data/ergodic_exp/trajectory.npy', allow_pickle=True)
for point in traj:
    marker_pose.pose.position.x = point[0] 
    marker_pose.pose.position.y = point[1]
    marker_pose.pose.position.z = point[2]
    rate.sleep()

In [ ]:
from mat2py.functions import pHRI_toyExample_PlanarManip01

rate = rospy.Rate(100)
currPos = matlab.double([[curr_Pos['x']], [curr_Pos['y']], [curr_Pos['z']]])
dt=np.arange(0.0, 2.0, 0.01 )
matlab_data = pHRI_toyExample_PlanarManip01(eng, currPos, dt)
matlab_data = np.squeeze(np.array(matlab_data))
trajectory_points = matlab_data.T[:, 1:4]

for point in trajectory_points:
    # print('X current:{}, desired:{}'.format(curr_Pos['x'], point[0]))
    # print('Z current:{}, desired:{}'.format(curr_Pos['z'], point[1]))
    marker_pose.pose.position.x = point[0] 
    marker_pose.pose.position.y = point[1]
    marker_pose.pose.position.z = point[2]
    rate.sleep()

### Test and compute 1 dt at a time with matlab

In [ ]:
from mat2py.functions import pHRI_toyExample_PlanarManip01
rate = rospy.Rate(50)

for dt in np.arange(0.0, 2.0, 0.02 ):
    currPos = matlab.double([[curr_Pos['x']], [curr_Pos['y']], [curr_Pos['z']]])
    matlab_data = pHRI_toyExample_PlanarManip01(eng, currPos, dt) # [t(1), currPos(2), currVel(2), currAcc(2), tarTmp(2), force(2)]
    matlab_data = np.squeeze(np.array(matlab_data))
    #matlab_data
    # print('X current:{}, desired:{}'.format(curr_Pos['x'], matlab_data[1]))
    # print('Z current:{}, desired:{}'.format(curr_Pos['z'], matlab_data[2]))
    #print(matlab_data[1:3])
    marker_pose.pose.position.x = matlab_data[1] 
    marker_pose.pose.position.z = matlab_data[2]
    rate.sleep()

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
import scipy.signal as signal
import numpy as np
from cycler import cycler

fig = plt.figure(figsize=plt.figaspect(0.5))
markercycle = cycler(marker=['o', '+', 'x', '*', '.', 'X'])
colorcycle = cycler(color=['blue', 'orange', 'green', 'magenta'])
# Or use the default color cycle:
# colorcycle = cycler(color=plt.rcParams['axes.prop_cycle'].by_key()['color'])

plt.gca().set_prop_cycle(colorcycle * markercycle[:4]) # gca()=current axis

ax = fig.add_subplot(1, 2, 1, projection ='3d')
data = Pn
resampled_data = signal.resample(data, 50)
ax.scatter(data[:,0], data[:,1], data[:,2])


ax = fig.add_subplot(1, 2, 2, projection ='3d')
ax.scatter(resampled_data[:,0], resampled_data[:,1], resampled_data[:,2])
ax.set_title('Test resempling')

plt.show()
#resampled_pos

In [ ]:
from mat2py.functions import pHRI_toyExample_PlanarManip01, toyExampleMSDstiffnessLearning

#pHRI_toyExample_PlanarManip01(eng)
dD = toyExampleMSDstiffnessLearning(eng)

### Data for collabrative2D paper

In [ ]:
eng.loader({'demo': [], 'xR1': start_pos}, 100)

In [ ]:
eng.quit()